In [ ]:
!pip install transformers tokenizers datasets

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
input_filename = "naslovi_new.input"
dataset_filename = input_filename + '.prepared'
pretrained_model_name = 'gpt2'
new_model_name = pretrained_model_name + '_4'

In [3]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast(tokenizer_file="/content/drive/MyDrive/ML-Vesti/naslovi-simple-tokenizer.json")
tokenizer.pad_token = "<pad>"


In [4]:
from datasets import load_dataset
# datasets = load_dataset('text', data_files=dataset_filename, split="train[90%],validate")
eval, train = load_dataset('text', data_files='/content/drive/MyDrive/ML-Vesti/' + dataset_filename, split=['train[:10%]', 'train[10%:]'])
def prepare_for_trainer(example):
  tokenized = tokenizer(example['text'], truncation=True, padding='max_length', max_length=20)
  tokenized["labels"] = tokenized["input_ids"].copy()
  return tokenized
tokenized_train = train.map(prepare_for_trainer, num_proc=2, remove_columns=["text"])
tokenized_eval = eval.map(prepare_for_trainer, num_proc=2, remove_columns=["text"])


Using custom data configuration default-9b16dadd29989af1
Reusing dataset text (/root/.cache/huggingface/datasets/text/default-9b16dadd29989af1/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)


Loading cached processed dataset at /root/.cache/huggingface/datasets/text/default-9b16dadd29989af1/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5/cache-08cefdf458753fe3.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/text/default-9b16dadd29989af1/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5/cache-7045a3e43f5a227b.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/text/default-9b16dadd29989af1/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5/cache-6d7f1153ad569538.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/text/default-9b16dadd29989af1/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5/cache-989d2cd40795db8a.arrow


In [5]:
from transformers import Trainer, AutoConfig, AutoModelForCausalLM, TrainingArguments

config = AutoConfig.from_pretrained(pretrained_model_name)
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name)
# model.to('cuda:0')
training_args = TrainingArguments(
    # new_model_name,
    output_dir='/content/drive/MyDrive/ML-Vesti/' + new_model_name,
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
    save_strategy='epoch',
    num_train_epochs=7,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    # prediction_loss_only=True,
    tokenizer=tokenizer,
)


In [6]:
import logging
logger = logging.getLogger(__name__)
logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        training_args.local_rank,
        training_args.device,
        training_args.n_gpu,
        bool(training_args.local_rank != -1),
        training_args.fp16,
    )

Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False


In [ ]:
# import gc
# import torch 
# model = None
# trainer = None
# gc.collect()
# torch.cuda.empty_cache()

In [7]:
train_result = trainer.train()
trainer.save_model('/content/drive/MyDrive/ML-Vesti/modeli/' + new_model_name)

***** Running training *****
  Num examples = 1968703
  Num Epochs = 7
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 107667


Epoch,Training Loss,Validation Loss
1,3.313200,3.908457
2,3.099800,3.663588
3,2.990500,3.543822
4,2.916900,3.472665
5,2.880600,3.431994
6,2.855900,3.407330
7,2.839900,3.398161


***** Running Evaluation *****
  Num examples = 218745
  Batch size = 128
Saving model checkpoint to /content/drive/MyDrive/ML-Vesti/gpt2_4/checkpoint-15381
Configuration saved in /content/drive/MyDrive/ML-Vesti/gpt2_4/checkpoint-15381/config.json
Model weights saved in /content/drive/MyDrive/ML-Vesti/gpt2_4/checkpoint-15381/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/ML-Vesti/gpt2_4/checkpoint-15381/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/ML-Vesti/gpt2_4/checkpoint-15381/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 218745
  Batch size = 128
Saving model checkpoint to /content/drive/MyDrive/ML-Vesti/gpt2_4/checkpoint-30762
Configuration saved in /content/drive/MyDrive/ML-Vesti/gpt2_4/checkpoint-30762/config.json
Model weights saved in /content/drive/MyDrive/ML-Vesti/gpt2_4/checkpoint-30762/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/ML-Vesti/gpt2_4/checkpoint-30762/token

In [8]:
import numpy as np
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained('/content/drive/MyDrive/ML-Vesti/modeli/' + new_model_name)
model.config.pad_token_id = 1

for temp in np.linspace(0.2, 2, 10):
    print()
    print("###### Temperature: " + str(temp))
    for i in range(0, 5):
        out = model.generate(do_sample=True, temperature=temp)[0]
        print(tokenizer.decode(out, skip_special_tokens=True))

loading configuration file /content/drive/MyDrive/ML-Vesti/modeli/gpt2_4/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.8.2",
  "use_cache": true,
  "vocab_size": 50257
}

loading weights file /content/drive/MyDrive/ML-Vesti/mode


###### Temperature: 0.2


!

je, a onda je! ( VIDEO )

###### Temperature: 0.4
Ovo su u Srbiji!
u :
,,,,,... dana

se, a onda je!!

###### Temperature: 0.6000000000000001
Posle toga što je rekao o, je u zatvoru!
je da je, ali je U!
Evo šta je sve pronađeno u
i u osmini finala, a evo kako su se na kraju
se na,, a onda je usledio šok!

###### Temperature: 0.8
Nakon svađe, otac i sin ( 14 ) pronašli u stanu
U Srbiji su se čule, a već je sve jasno
U na i GODINE GODINE!
je da su joj se!, pa je! ( VIDEO )
, i dobili za

###### Temperature: 1.0
Bivši trener Zvezde OTKRIO plan koji je oduševio Evropu ( FOTO )
Vučić u Vašingtonu sa : nije opcija, ne postoje stvari
Nakon ŽESTOKE svađe sa Borom PEVAČICOM, ONA je jedna stvar za to!
me, nije ni znao gde će!
Ovo je pet albuma koje bi trebalo da preslušate danas ( FOTO )

###### Temperature: 1.2
Ana Nikolić : Zašto je i gde živi, što svi kažu da ćemo to da radite
U udesu motocikla autobusa oko 25 povređeno! Povređena 3 pešaka!
Pevačica se opustila u

In [9]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-a94002ea-f93f-02c5-c7a3-6842be217f8f)
